<span style="color:red"><b>  생각해보기  </b></span>  <br>
* data에 outlier에 대해서도 해야하니 quantile regression 이 적합?? <br>
    -> probability 처럼 나타낼 수도 있음 <br>
    
    
* 바람 쌘 날이 잘 반영되지 않음 <br>
    -> 어떤 새로운 변수를 만들어야 하나??<br>
    
    
* TMA Capacity 정의 <br>
    TMA와 Aiport의 Capacity에 차이라고 한다면 holding으로 인한 것 뿐인 것 같음 <br>
    -> 이 부분만 airport capacity에 포함을 시키면 되지않을까? <br>
    
  
* 이 ML이 prediction이라고 주는 값은,  Demand가 ~고, 날씨가 ~일 떄 몇대가 들어올 것 같다는 값을 줌 <br>
    -> 이 항공기 숫자를 robust하게 예측하게 하고(확률적으로) reservior 개념을 적용해서 scheduling problem으로 발전(holding은 어떻게 넣을래?)  <br>
    
    
* 
<span style="color:red"> *** </span>
<b> reservoir를 하려면 AAR, ADR을 STAR, SID에 들어오는 수로 해야겠는데??? (특히 Arrival) </b> <br>
    그래야지 TMA용량 정의를 holding하는 항공기 수를 포함하여 적용가능<br>
    departure는 거의 공항과 같게 될 것 같음(departure는 홀딩을 거의 공항에서 하므로) , 아니면 departure로 hand off되는 고도에 도착하는 걸로
<br>
<br>
<br>
***
<br>    
<br>    
<br>
예를들어, reservoir에서 15분까지 airborne holding까지 허용한다고 하자 <br>
지금 방법으로하면 Demand만 한대씩 늘려가면서, 몇 대가 들어올 지 예측해서 값을 주는 거지 <br>
(즉, A대가 그떄 도착하도록 이렇게저렇게 GDP를 조정해서 보낸다 하면, 그 중 몇대가 B대가 그시간대에 올지 예측해서 값으로 줌) <br>
-> 이렇게 한대씩 늘렸을 때 예측값으로 준 rate과 불확실성을 대변하는 무엇(별표)을 가지고 최대 15분 holding이라는 기준을 맞추도록 stochastic scheduling을 해서 최종적으로 적절하게 보내도 되는 A값을 찾음 (여러 곳에서 오는 신호를 순서대로 처리하라고 보내는 것이라고 할 수도 있음)  <br>
-> A값을 그 상황에서의 Capacity 로 정의 <br>
<br>
<br>
<br>
별표 <br>
몇 대가 올지 예측한 거를 확률로?<br>
근데 어디서 올지, 언제 올지를 모른단 말이지 -> 이것도 확률로??<br>
***

# Data Preparation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import optuna.integration.lightgbm as lgb          ####
from lightgbm import LGBMRegressor, plot_metric    #### 여기서 lgb로 안가져옴
from pathlib import Path
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold

In [2]:
# options
pd.set_option('max_columns',100)
plt.style.use('fivethirtyeight')
warnings.simplefilter('ignore')
seed = 42

In [3]:
# Data dirctory
data_dir = Path('../Terminal_Capacity/Data/')
data_file = data_dir / 'Data.csv'

In [4]:
# Data
# 0:AAR / 1:EAD / 2:ADR / 3:EDD는 고정  , 나머지는 순서 상관 없음
Data = pd.read_csv(data_file, index_col=0)
ColumnName = Data.columns
Data.head(30)

,AAR,EAD,ADR,EDD,year,month,day,hour,DayName,Arpt_cond,P_Airp,P_AAR,P_ADR,Arrival_remainder,Departure_remainder,WD_400,WD_500,WD_700,WD_850,WD_925,WD_1000,WS_400,WS_500,WS_700,WS_850,WS_925,WS_1000,WD,WSPD,WS_GST,VIS,WC,TMP,TD,PS,PA,RN,HM,CA_TOT,CLA_1LYR,BASE_1LYR,CLA_2LYR,BASE_2LYR,CLA_3LYR,BASE_3LYR,CLA_4LYR,BASE_4LYR,RVR,WDIR_t6,WSPD_t6,WG_t6,VIS_t6,WC_t6,CLA_1LYR_t6,BASE_1LYR_t6,CLA_2LYR_t6,BASE_2LYR_t6,CLA_3LYR_t6,BASE_3LYR_t6,WDIR_t12,WSPD_t12,WG_t12,VIS_t12,WC_t12,CLA_1LYR_t12,BASE_1LYR_t12,CLA_2LYR_t12,BASE_2LYR_t12,CLA_3LYR_t12,BASE_3LYR_t12,WDIR_t18,WSPD_t18,WG_t18,VIS_t18,WC_t18,CLA_1LYR_t18,BASE_1LYR_t18,CLA_2LYR_t18,BASE_2LYR_t18,CLA_3LYR_t18,BASE_3LYR_t18,WDIR_t24,WSPD_t24,WG_t24,VIS_t24,WC_t24,CLA_1LYR_t24,BASE_1LYR_t24,CLA_2LYR_t24,BASE_2LYR_t24,CLA_3LYR_t24,BASE_3LYR_t24
0,4.0,3.0,7.0,9.0,2019,1,1,0,2,1.0,1.0,0.0,0.0,0.0,0.0,275.0,265.0,245.0,280.0,280.0,305.0,96.0,80.0,31.0,12.0,23.0,8.0,1,6,0.0,1000,1,-53,-102,10359,10350,0.0,68,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
1,2.0,3.0,6.0,7.0,2019,1,1,1,2,1.0,1.0,4.0,7.0,0.0,2.0,275.0,265.0,245.0,280.0,280.0,305.0,96.0,80.0,31.0,12.0,23.0,8.0,31,5,0.0,1000,1,-57,-105,10358,10349,0.0,69,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
2,0.0,2.0,3.0,3.0,2019,1,1,2,2,1.0,1.0,2.0,6.0,1.0,1.0,275.0,265.0,245.0,280.0,280.0,305.0,96.0,80.0,31.0,12.0,23.0,8.0,33,4,0.0,1000,1,-60,-105,10359,10350,0.0,70,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
3,6.0,1.0,5.0,2.0,2019,1,1,3,2,1.0,1.0,0.0,3.0,2.0,0.0,275.0,265.0,245.0,280.0,280.0,305.0,96.0,80.0,31.0,12.0,23.0,8.0,32,4,0.0,1000,1,-63,-112,10363,10354,0.0,68,0,0.0,400.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
4,20.0,15.0,1.0,2.0,2019,1,1,4,2,1.0,1.0,6.0,5.0,0.0,0.0,275.0,265.0,245.0,280.0,280.0,305.0,96.0,80.0,31.0,12.0,23.0,8.0,7,1,0.0,1000,1,-66,-118,10357,10348,0.0,66,2,2.0,30.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
5,31.0,36.0,1.0,1.0,2019,1,1,5,2,1.0,1.0,20.0,1.0,0.0,1.0,275.0,265.0,245.0,280.0,280.0,305.0,96.0,80.0,31.0,12.0,23.0,8.0,4,1,0.0,1000,1,-71,-115,10353,10344,0.0,70,2,2.0,30.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
6,35.0,36.0,6.0,16.0,2019,1,1,6,2,1.0,1.0,31.0,1.0,5.0,0.0,275.0,275.0,280.0,290.0,295.0,290.0,121.0,102.0,27.0,26.0,18.0,9.0,2,3,0.0,1000,1,-77,-120,10349,10340,0.0,71,2,2.0,30.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
7,29.0,27.0,27.0,30.0,2019,1,1,7,2,1.0,1.0,35.0,6.0,1.0,10.0,275.0,275.0,280.0,290.0,295.0,290.0,121.0,102.0,27.0,26.0,18.0,9.0,7,3,0.0,1000,1,-76,-118,10346,10337,0.0,71,2,2.0,30.0,0.0,400.0,0.0,400.0,0.0,400.0,1000.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0,0.0,0.0,0.0,9999.0,0,0,400.0,0,400.0,0,400.0
8,22.0,23.0,33.0,32.0,2019,1,1,8,2,1.0,1.0,29.0

***
***
***



##  왜?

In [ ]:
Data['AAR'].value_counts()

In [ ]:
pred = pd.DataFrame({'pred' :reg_arrival.predict(Data_a)})
print('9. : ', pred[(pred['pred'] < 10) & (pred['pred'] >= 9)].shape)
print('10. : ', pred[(pred['pred'] < 11) & (pred['pred'] >= 10)].shape)
print('11. : ', pred[(pred['pred'] < 12) & (pred['pred'] >= 11)].shape)
print('12. : ', pred[(pred['pred'] < 13) & (pred['pred'] >= 12)].shape,'*')
print('13. : ', pred[(pred['pred'] < 14) & (pred['pred'] >= 13)].shape,'*')
print('14. : ', pred[(pred['pred'] < 15) & (pred['pred'] >= 14)].shape,'*')
print('15. : ', pred[(pred['pred'] < 16) & (pred['pred'] >= 15)].shape,'*')
print('16. : ', pred[(pred['pred'] < 17) & (pred['pred'] >= 16)].shape)
print('17. : ', pred[(pred['pred'] < 18) & (pred['pred'] >= 17)].shape)

In [ ]:
sssss = pred[(pred['pred'] < 16) & (pred['pred'] >= 12)].index
Data.loc[sssss]

In [ ]:
ddddd = Data[(Data['AAR'] >= 12)&(Data['AAR']<15)].index

DD = Data[(Data['AAR'] >= 12)&(Data['AAR']<=16)]
DDDD = pred.loc[ddddd].join(DD).head(30)

In [ ]:
sns.pairplot(data=DDDD, vars=['AAR','pred'], size=6, plot_kws={'alpha': 1})

In [ ]:
DDDD.describe()

In [ ]:
fea = 'WD'

sns.distplot(DDDD[fea])

print("AAR Skewness: %f" % Data[fea].skew())
print("ADR Kurtosis: %f" % Data[fea].kurt())

# P_Airp Arrival_remainder

***
***
***

# Data Preprocessing

In [ ]:
# Feature Scaling

# decision tree류의 알고리즘은 Scaling(standardization, normalization)이 큰 의미 X


"""
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()
X = scaler.fit_transform(df[num_cols])
lr = LinearRegression()
lr.fit(X, np.log1p(df[target_col]))
df[pred_col] = np.expm1(lr.predict(X))

scaler = MinMaxScaler()
X = scaler.fit_transform(df[num_cols])
lr = LinearRegression()
lr.fit(X, np.log1p(df[target_col]))
df[pred_col] = np.expm1(lr.predict(X))
"""

In [ ]:
#Binning

#어떤 feature를 n개의 그룹으로 나누고, 그것을 새로운 categorical data로 넣는 것

"""
df['time_bin'] = pd.qcut(df['time'], 4, labels=False)
sns.pairplot(data=df, vars=['time', 'time_bin'], size=4, plot_kws={'alpha': .5})


X = pd.concat([df[num_cols], pd.get_dummies(pd.qcut(df['time'], 4, labels=False))], axis=1)    
# get_dummies 는 one-hot encoding해주는 것(decision tree 계열을 안 하는게 보통 더 좋은 결과를 냄)

lr = LinearRegression()
lr.fit(X, np.log1p(df[target_col]))
df[pred_col] = np.expm1(lr.predict(X))
"""

In [ ]:
# Polynomial Regression

# 연속적인 몇개의 feature들을 조합해서 새로운 feature를 만드는 것 (overfitting 위험 O)

"""
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)        # 2차(x^2, x1 * x2 등) 까지만 만들겠다
X = poly.fit_transform(df[num_cols])

lr = LinearRegression()
lr.fit(X, np.log1p(df[target_col]))
df[pred_col] = np.expm1(lr.predict(X))
"""

In [ ]:
# Checking for NaN values and removing constant features in the training data
# Removing duplicated columns
# Drop Sparse Data

### Add Features
# Sumzeros and Sumvalues
# Other Aggregates
# K-Means
# PCA : Principal component analysis



***
***
***

# Data Selection

In [5]:
# 필요없는 것을 버리기
Data_temp = Data.drop('TMP', axis=1)
Data_temp = Data_temp.drop('TD', axis=1)
Data_temp = Data_temp.drop('HM', axis=1)
Data_temp = Data_temp.drop('PS', axis=1)
Data_temp = Data_temp.drop('PA', axis=1)

######## 고층바람이 생각보다 영향이 크다?? -> 너무 높은 고도는 뺴고 해보자  ########
Data_temp = Data_temp.drop('WD_400', axis=1)
Data_temp = Data_temp.drop('WD_500', axis=1)
Data_temp = Data_temp.drop('WD_700', axis=1)
Data_temp = Data_temp.drop('WS_400', axis=1)
Data_temp = Data_temp.drop('WS_500', axis=1)
Data_temp = Data_temp.drop('WS_700', axis=1)



taf6 = [12,18,24]
taf12 = [6,18,24]
taf18 = [6,12,24]
taf24 = [6,12,18]

# drop TAF
for i in range(6,30,6):
    Data_temp = Data_temp.drop(f'WDIR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WSPD_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WG_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'VIS_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WC_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_1LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_1LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_2LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_2LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_3LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_3LYR_t{i}', axis=1)

In [6]:
# 각 시간에 맞는 taf 넣기

data_taf = {}
for i in range(6,30,6):
    data_taf[f'Data_{i}'] = Data_temp    
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WDIR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WSPD_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WG_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'VIS_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WC_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_1LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_1LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_2LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_2LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_3LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_3LYR_t{i}'])

In [7]:
Data_6 = data_taf['Data_6']
Data_12 = data_taf['Data_12']
Data_18 = data_taf['Data_18']
Data_24 = data_taf['Data_24']

Validation <br><br>

sklearn.model_selection
* train_test_split
* KFold
* StratifiedKFold

<br>
<br>
lightGBM <br>
* lgb.cv    # 쓰려면 lightgbm을 lgb로 import

***
***
***

# LightGBM fitting

In [8]:
# Hyperparameters
params = {'boosting_type' : 'gbdt',            # 'dart' 는 계산시간 길어짐, early stopping X / 'rf’ : Random Forest
          'metric': 'mse',
          'num_leaves' : 127,                  ## Maximum tree leaves for base learners (31)
          'max_depth' : - 1,                   # Maximum tree depth for base learners, <=0 means no limit (-1)
          'learning_rate' : 0.001,             ## Boosting learning rate (0.1)
          'n_estimators' : 10000000,           # Number of boosted trees to fit (100) -> fit에서 early stopping으로 제한해서 크게 설정함
          'subsample_for_bin' : 200000,        # Number of samples for constructing bins (200000)
          'objective' : 'regression',          # learning task and the corresponding learning objective (None)
          'class_weight' : None,               # * Use this parameter only for multi-class classification task
          'min_split_gain' : 0.0,              # Minimum loss reduction required to make a further partition on a leaf node of the tree (0)
          'min_child_weight' : 0.001,          # Minimum sum of instance weight (hessian) needed in a child (leaf) (0.001)
          'min_child_samples' : 1,             # Minimum number of data needed in a child (leaf) (20) - 마지막노드(리프)에 최소 몇가지 샘플이 있어야 하는지 
          'subsample' : 0.8,                   ## Subsample ratio of the training instance (1.0) - 개별 트리를 학습시키는데 몇 %의 데이터를 사용할 것 인지, row sampling
          'subsample_freq' : 1,                # Frequency of subsample, <=0 means no enable (0) - 몇개의 트리마다 subsampling을 할 것인지
          'colsample_bytree' : 0.8,            ## Subsample ratio of columns when constructing each tree (1.0) - 몇 %의 column을 sampling 할 것인지
          'reg_alpha' : 0.0,                   # L1 regularization term on weights (0)
          'reg_lambda' : 0.0,                  # L2 regularization term on weights (0)
          'random_state' : seed,               # Random number seed (None)
          'n_jobs' : - 1,                      # Number of parallel threads (-1) - 몇 개의 병렬작업을 할 것인지 (-1 = 모든 가능한 것 전부)
          'silent' : True,                     # Whether to print messages while running boosting (True)
          'importance_type' : 'split'}         # ‘split’: result contains numbers of times the feature is used in a model
                                               # ‘gain’ : result contains total gains of splits which use the feature

# 최적 hyperparameter 찾기!!

In [9]:
# Hyperparameters
params_a = {'boosting_type' : 'gbdt',                   # 'dart' 는 계산시간 길어짐, early stopping X / 'rf’ : Random Forest
            'metric': 'mse',
            'num_leaves' : 127,                          ## Maximum tree leaves for base learners (31)
            'max_depth' : - 1,                          # Maximum tree depth for base learners, <=0 means no limit (-1)
            'learning_rate' : 0.001,                    ## Boosting learning rate (0.1)
            'n_estimators' : 10000000,                  # Number of boosted trees to fit (100) -> fit에서 early stopping으로 제한해서 크게 설정함
            'subsample_for_bin' : 200000,               # Number of samples for constructing bins (200000)
            'objective' : 'regression',                 # learning task and the corresponding learning objective (None)
            'class_weight' : None,                      # * Use this parameter only for multi-class classification task
            'min_split_gain' : 0.0,                     # Minimum loss reduction required to make a further partition on a leaf node of the tree (0)
            'min_child_weight' : 0.001,                 # Minimum sum of instance weight (hessian) needed in a child (leaf) (0.001)
            'min_child_samples' : 1,                    # Minimum number of data needed in a child (leaf) (20) - 마지막노드(리프)에 최소 몇가지 샘플이 있어야 하는지 
            'feature_pre_filter': False,
            'subsample' : 0.8941758297667703,           ## Subsample ratio of the training instance (1.0) - 개별 트리를 학습시키는데 몇 %의 데이터를 사용할 것 인지, row sampling
            'subsample_freq' : 1,   #3                  # Frequency of subsample, <=0 means no enable (0) - 몇개의 트리마다 subsampling을 할 것인지
            'colsample_bytree' : 0.7520000000000001,    ## Subsample ratio of columns when constructing each tree (1.0) - 몇 %의 column을 sampling 할 것인지
            'reg_alpha' : 0.00010798877450667029,       # L1 regularization term on weights (0)
            'reg_lambda' : 0.572291122965791,           # L2 regularization term on weights (0)
            'random_state' : seed,                      # Random number seed (None)
            'n_jobs' : - 1,                             # Number of parallel threads (-1) - 몇 개의 병렬작업을 할 것인지 (-1 = 모든 가능한 것 전부)
            'silent' : True,                            # Whether to print messages while running boosting (True)
            'importance_type' : 'split'}                # ‘split’: result contains numbers of times the feature is used in a model
                                                        # ‘gain’ : result contains total gains of splits which use the feature

In [10]:
# Hyperparameters
params_d = {'boosting_type' : 'gbdt',                 # 'dart' 는 계산시간 길어짐, early stopping X / 'rf’ : Random Forest
            'metric': 'mse',
            'num_leaves' : 127,                        ## Maximum tree leaves for base learners (31)
            'max_depth' : - 1,                        # Maximum tree depth for base learners, <=0 means no limit (-1)
            'learning_rate' : 0.001,                  ## Boosting learning rate (0.1)
            'n_estimators' : 10000000,                # Number of boosted trees to fit (100) -> fit에서 early stopping으로 제한해서 크게 설정함
            'subsample_for_bin' : 200000,             # Number of samples for constructing bins (200000)
            'objective' : 'regression',               # learning task and the corresponding learning objective (None)
            'class_weight' : None,                    # * Use this parameter only for multi-class classification task
            'min_split_gain' : 0.0,                   # Minimum loss reduction required to make a further partition on a leaf node of the tree (0)
            'min_child_weight' : 0.001,               # Minimum sum of instance weight (hessian) needed in a child (leaf) (0.001)
            'min_child_samples' : 1,                  # Minimum number of data needed in a child (leaf) (20) - 마지막노드(리프)에 최소 몇가지 샘플이 있어야 하는지 
            'feature_pre_filter': False,
            'subsample' : 0.8862849544828171,         ## Subsample ratio of the training instance (1.0) - 개별 트리를 학습시키는데 몇 %의 데이터를 사용할 것 인지, row sampling
            'subsample_freq' : 1,    #3               # Frequency of subsample, <=0 means no enable (0) - 몇개의 트리마다 subsampling을 할 것인지
            'colsample_bytree' : 0.7,                 ## Subsample ratio of columns when constructing each tree (1.0) - 몇 %의 column을 sampling 할 것인지
            'reg_alpha' : 3.2041046397019426e-05,     # L1 regularization term on weights (0)
            'reg_lambda' : 2.4947477155359143e-05,    # L2 regularization term on weights (0)
            'random_state' : seed,                    # Random number seed (None)
            'n_jobs' : - 1,                           # Number of parallel threads (-1) - 몇 개의 병렬작업을 할 것인지 (-1 = 모든 가능한 것 전부)
            'silent' : True,                          # Whether to print messages while running boosting (True)
            'importance_type' : 'split'}              # ‘split’: result contains numbers of times the feature is used in a model
                                                      # ‘gain’ : result contains total gains of splits which use the feature

In [11]:
# 예측할 시간에 맞는 Data로 넣기
# 0-6 : Data_6 / 6-12 : Data_12 / 12-18 : Data_18 / 18-24 : Data_24

Data_m = Data_6
Data_raw = Data_6

***
## Hyperparameter Optimization

In [ ]:
# Optuna 간단 버전 - arrival

"""

Data_a = Data_m.drop('AAR', axis=1)
y = Data_raw.AAR.to_numpy()
X = Data_a.to_numpy()

X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X, y, test_size=.2, random_state=seed)

dtrain_a = lgb.Dataset(X_train_a, label = y_train_a)
dval_a = lgb.Dataset(X_val_a, label = y_val_a)

model = lgb.train(params, dtrain_a,
                  valid_sets=[dtrain_a, dval_a], 
                  verbose_eval=100,
                  early_stopping_rounds=10)

prediction = model.predict(X_val_a, num_iteration=model.best_iteration)

accuracy = r2_score(y_val_a, prediction)        # classification이면 앞에 argmax( , axis = 1)로 하면 될 듯



params = model.params
print("Best params:", params)
print("  Accuracy = {}".format(accuracy))o
print("  Params: ")
for key, value in params.items():
    print("    {}: {}".format(key, value))

params_a = params

"""

In [ ]:
# Optuna 간단 버전 - departure

"""

Data_d = Data_m.drop('ADR', axis=1)
y = Data_raw.ADR.to_numpy()
X = Data_d.to_numpy()

X_train_d, X_val_d, y_train_d, y_val_d = train_test_split(X, y, test_size=.2, random_state=seed)

dtrain_d = lgb.Dataset(X_train_d, label = y_train_d)
dval_d = lgb.Dataset(X_val_d, label = y_val_d)

model = lgb.train(params, dtrain_d,
                  valid_sets=[dtrain_d, dval_d], 
                  verbose_eval=100,
                  early_stopping_rounds=10)

prediction = model.predict(X_val_d, num_iteration=model.best_iteration)

accuracy = r2_score(y_val_d, prediction)         # classification이면 앞에 argmax( , axis = 1)로 하면 될 듯

params = model.params
print("Best params:", params)
print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in params.items():
    print("    {}: {}".format(key, value))

params_d = params

"""

***
## Arrival

In [14]:
Data_a = Data_m.drop('AAR', axis=1)
Data_a = Data_a.drop('ADR', axis=1)
y = Data_raw.AAR.to_numpy()
X = Data_a.to_numpy()

#X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X, y, test_size = 0.1, random_state = seed)
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X, y, test_size = 0.1, random_state = seed)
X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X_train_a, y_train_a, test_size=0.11, random_state = 13) # 0.11 x 0.9 = 0.099

In [15]:
# Model fitting
reg_arrival = LGBMRegressor(**params_a)

reg_arrival.fit(X_train_a, y_train_a,
                sample_weight = None,                   # Weights of training data
                init_score = None,                      # Weights of training data
                eval_set = [(X_val_a, y_val_a)],        # pairs to use as validation sets
                eval_sample_weight = None,              # Weights of eval data
                eval_init_score = None,                 # Init score of eval data.   
                eval_metric = 'l2',                     # Default: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier, ‘ndcg’ for LGBMRanker
                early_stopping_rounds = 10 )            # loss fuction이 n번 이상 좋아지지 않으면 멈춰라

[1]	valid_0's l2: 123.617
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 123.395
[3]	valid_0's l2: 123.17
[4]	valid_0's l2: 122.949
[5]	valid_0's l2: 122.744
[6]	valid_0's l2: 122.52
[7]	valid_0's l2: 122.299
[8]	valid_0's l2: 122.078
[9]	valid_0's l2: 121.858
[10]	valid_0's l2: 121.637
[11]	valid_0's l2: 121.416
[12]	valid_0's l2: 121.197
[13]	valid_0's l2: 120.975
[14]	valid_0's l2: 120.753
[15]	valid_0's l2: 120.531
[16]	valid_0's l2: 120.313
[17]	valid_0's l2: 120.095
[18]	valid_0's l2: 119.876
[19]	valid_0's l2: 119.662
[20]	valid_0's l2: 119.442
[21]	valid_0's l2: 119.225
[22]	valid_0's l2: 119.007
[23]	valid_0's l2: 118.79
[24]	valid_0's l2: 118.573
[25]	valid_0's l2: 118.358
[26]	valid_0's l2: 118.142
[27]	valid_0's l2: 117.926
[28]	valid_0's l2: 117.711
[29]	valid_0's l2: 117.515
[30]	valid_0's l2: 117.301
[31]	valid_0's l2: 117.088
[32]	valid_0's l2: 116.895
[33]	valid_0's l2: 116.686
[34]	valid_0's l2: 116.532
[35]	valid_0's l2: 116.324
[36]	v

[299]	valid_0's l2: 73.0018
[300]	valid_0's l2: 72.8754
[301]	valid_0's l2: 72.7493
[302]	valid_0's l2: 72.6242
[303]	valid_0's l2: 72.4975
[304]	valid_0's l2: 72.3735
[305]	valid_0's l2: 72.2483
[306]	valid_0's l2: 72.1246
[307]	valid_0's l2: 72.0007
[308]	valid_0's l2: 71.8969
[309]	valid_0's l2: 71.7719
[310]	valid_0's l2: 71.6475
[311]	valid_0's l2: 71.5253
[312]	valid_0's l2: 71.4013
[313]	valid_0's l2: 71.2779
[314]	valid_0's l2: 71.1552
[315]	valid_0's l2: 71.0323
[316]	valid_0's l2: 70.9217
[317]	valid_0's l2: 70.8099
[318]	valid_0's l2: 70.6874
[319]	valid_0's l2: 70.5648
[320]	valid_0's l2: 70.4452
[321]	valid_0's l2: 70.3241
[322]	valid_0's l2: 70.2056
[323]	valid_0's l2: 70.0836
[324]	valid_0's l2: 69.9656
[325]	valid_0's l2: 69.8447
[326]	valid_0's l2: 69.7245
[327]	valid_0's l2: 69.6324
[328]	valid_0's l2: 69.5828
[329]	valid_0's l2: 69.4627
[330]	valid_0's l2: 69.3432
[331]	valid_0's l2: 69.2259
[332]	valid_0's l2: 69.106
[333]	valid_0's l2: 68.9864
[334]	valid_0's l2: 6

[593]	valid_0's l2: 45.1278
[594]	valid_0's l2: 45.0577
[595]	valid_0's l2: 44.9883
[596]	valid_0's l2: 44.9168
[597]	valid_0's l2: 44.8449
[598]	valid_0's l2: 44.7748
[599]	valid_0's l2: 44.7057
[600]	valid_0's l2: 44.6355
[601]	valid_0's l2: 44.5644
[602]	valid_0's l2: 44.495
[603]	valid_0's l2: 44.4305
[604]	valid_0's l2: 44.3615
[605]	valid_0's l2: 44.2914
[606]	valid_0's l2: 44.2213
[607]	valid_0's l2: 44.1511
[608]	valid_0's l2: 44.0805
[609]	valid_0's l2: 44.0113
[610]	valid_0's l2: 43.9425
[611]	valid_0's l2: 43.874
[612]	valid_0's l2: 43.806
[613]	valid_0's l2: 43.7384
[614]	valid_0's l2: 43.6713
[615]	valid_0's l2: 43.6039
[616]	valid_0's l2: 43.5344
[617]	valid_0's l2: 43.4672
[618]	valid_0's l2: 43.3985
[619]	valid_0's l2: 43.3312
[620]	valid_0's l2: 43.2644
[621]	valid_0's l2: 43.196
[622]	valid_0's l2: 43.1282
[623]	valid_0's l2: 43.0605
[624]	valid_0's l2: 42.995
[625]	valid_0's l2: 42.9283
[626]	valid_0's l2: 42.8611
[627]	valid_0's l2: 42.7942
[628]	valid_0's l2: 42.72

[897]	valid_0's l2: 28.8815
[898]	valid_0's l2: 28.8422
[899]	valid_0's l2: 28.8021
[900]	valid_0's l2: 28.762
[901]	valid_0's l2: 28.7228
[902]	valid_0's l2: 28.6835
[903]	valid_0's l2: 28.6443
[904]	valid_0's l2: 28.6051
[905]	valid_0's l2: 28.5659
[906]	valid_0's l2: 28.531
[907]	valid_0's l2: 28.4932
[908]	valid_0's l2: 28.4536
[909]	valid_0's l2: 28.4147
[910]	valid_0's l2: 28.3757
[911]	valid_0's l2: 28.3363
[912]	valid_0's l2: 28.2972
[913]	valid_0's l2: 28.2581
[914]	valid_0's l2: 28.2199
[915]	valid_0's l2: 28.1821
[916]	valid_0's l2: 28.1444
[917]	valid_0's l2: 28.1077
[918]	valid_0's l2: 28.0691
[919]	valid_0's l2: 28.0307
[920]	valid_0's l2: 27.9921
[921]	valid_0's l2: 27.954
[922]	valid_0's l2: 27.9173
[923]	valid_0's l2: 27.88
[924]	valid_0's l2: 27.8428
[925]	valid_0's l2: 27.8059
[926]	valid_0's l2: 27.7686
[927]	valid_0's l2: 27.7311
[928]	valid_0's l2: 27.6931
[929]	valid_0's l2: 27.6559
[930]	valid_0's l2: 27.618
[931]	valid_0's l2: 27.5806
[932]	valid_0's l2: 27.544

[1190]	valid_0's l2: 20.0686
[1191]	valid_0's l2: 20.0456
[1192]	valid_0's l2: 20.0234
[1193]	valid_0's l2: 20.0016
[1194]	valid_0's l2: 19.9787
[1195]	valid_0's l2: 19.956
[1196]	valid_0's l2: 19.9338
[1197]	valid_0's l2: 19.9114
[1198]	valid_0's l2: 19.8898
[1199]	valid_0's l2: 19.8678
[1200]	valid_0's l2: 19.8457
[1201]	valid_0's l2: 19.8234
[1202]	valid_0's l2: 19.8035
[1203]	valid_0's l2: 19.7815
[1204]	valid_0's l2: 19.7599
[1205]	valid_0's l2: 19.7377
[1206]	valid_0's l2: 19.7157
[1207]	valid_0's l2: 19.6936
[1208]	valid_0's l2: 19.672
[1209]	valid_0's l2: 19.6514
[1210]	valid_0's l2: 19.6292
[1211]	valid_0's l2: 19.6071
[1212]	valid_0's l2: 19.5853
[1213]	valid_0's l2: 19.5631
[1214]	valid_0's l2: 19.5413
[1215]	valid_0's l2: 19.52
[1216]	valid_0's l2: 19.501
[1217]	valid_0's l2: 19.4793
[1218]	valid_0's l2: 19.4575
[1219]	valid_0's l2: 19.4362
[1220]	valid_0's l2: 19.4144
[1221]	valid_0's l2: 19.3935
[1222]	valid_0's l2: 19.3728
[1223]	valid_0's l2: 19.3519
[1224]	valid_0's l2

...

[3303]	valid_0's l2: 8.3847
[3304]	valid_0's l2: 8.38421
[3305]	valid_0's l2: 8.38376
[3306]	valid_0's l2: 8.38338
[3307]	valid_0's l2: 8.38286
[3308]	valid_0's l2: 8.38214
[3309]	valid_0's l2: 8.38156
[3310]	valid_0's l2: 8.38104
[3311]	valid_0's l2: 8.38085
[3312]	valid_0's l2: 8.38012
[3313]	valid_0's l2: 8.37962
[3314]	valid_0's l2: 8.37888
[3315]	valid_0's l2: 8.37852
[3316]	valid_0's l2: 8.37842
[3317]	valid_0's l2: 8.37791
[3318]	valid_0's l2: 8.37745
[3319]	valid_0's l2: 8.37691
[3320]	valid_0's l2: 8.376
[3321]	valid_0's l2: 8.37516
[3322]	valid_0's l2: 8.37465
[3323]	valid_0's l2: 8.37428
[3324]	valid_0's l2: 8.37378
[3325]	valid_0's l2: 8.37343
[3326]	valid_0's l2: 8.37285
[3327]	valid_0's l2: 8.37234
[3328]	valid_0's l2: 8.3714
[3329]	valid_0's l2: 8.37084
[3330]	valid_0's l2: 8.37029
[3331]	valid_0's l2: 8.36962
[3332]	valid_0's l2: 8.36946
[3333]	valid_0's l2: 8.36873
[3334]	valid_0's l2: 8.36847
[3335]	valid_0's l2: 8.36805
[3336]	valid_0's l2: 8.36769
[3337]	valid_0's l

[3590]	valid_0's l2: 8.27364
[3591]	valid_0's l2: 8.2733
[3592]	valid_0's l2: 8.27314
[3593]	valid_0's l2: 8.27282
[3594]	valid_0's l2: 8.2728
[3595]	valid_0's l2: 8.27221
[3596]	valid_0's l2: 8.27181
[3597]	valid_0's l2: 8.27143
[3598]	valid_0's l2: 8.27107
[3599]	valid_0's l2: 8.2706
[3600]	valid_0's l2: 8.27036
[3601]	valid_0's l2: 8.27014
[3602]	valid_0's l2: 8.26983
[3603]	valid_0's l2: 8.26947
[3604]	valid_0's l2: 8.26911
[3605]	valid_0's l2: 8.269
[3606]	valid_0's l2: 8.26851
[3607]	valid_0's l2: 8.2681
[3608]	valid_0's l2: 8.26774
[3609]	valid_0's l2: 8.26754
[3610]	valid_0's l2: 8.26704
[3611]	valid_0's l2: 8.26654
[3612]	valid_0's l2: 8.2662
[3613]	valid_0's l2: 8.26583
[3614]	valid_0's l2: 8.26574
[3615]	valid_0's l2: 8.26521
[3616]	valid_0's l2: 8.26494
[3617]	valid_0's l2: 8.26433
[3618]	valid_0's l2: 8.264
[3619]	valid_0's l2: 8.2636
[3620]	valid_0's l2: 8.26338
[3621]	valid_0's l2: 8.26288
[3622]	valid_0's l2: 8.26244
[3623]	valid_0's l2: 8.26211
[3624]	valid_0's l2: 8.2

[3905]	valid_0's l2: 8.19643
[3906]	valid_0's l2: 8.19606
[3907]	valid_0's l2: 8.1957
[3908]	valid_0's l2: 8.19517
[3909]	valid_0's l2: 8.19478
[3910]	valid_0's l2: 8.19466
[3911]	valid_0's l2: 8.19451
[3912]	valid_0's l2: 8.19436
[3913]	valid_0's l2: 8.19419
[3914]	valid_0's l2: 8.19413
[3915]	valid_0's l2: 8.19379
[3916]	valid_0's l2: 8.19392
[3917]	valid_0's l2: 8.19381
[3918]	valid_0's l2: 8.19349
[3919]	valid_0's l2: 8.19345
[3920]	valid_0's l2: 8.19336
[3921]	valid_0's l2: 8.19294
[3922]	valid_0's l2: 8.19288
[3923]	valid_0's l2: 8.19282
[3924]	valid_0's l2: 8.19272
[3925]	valid_0's l2: 8.1929
[3926]	valid_0's l2: 8.19266
[3927]	valid_0's l2: 8.19247
[3928]	valid_0's l2: 8.19233
[3929]	valid_0's l2: 8.19212
[3930]	valid_0's l2: 8.19166
[3931]	valid_0's l2: 8.19135
[3932]	valid_0's l2: 8.19099
[3933]	valid_0's l2: 8.19056
[3934]	valid_0's l2: 8.19049
[3935]	valid_0's l2: 8.19039
[3936]	valid_0's l2: 8.19026
[3937]	valid_0's l2: 8.19021
[3938]	valid_0's l2: 8.18976
[3939]	valid_0's

[4190]	valid_0's l2: 8.15225
[4191]	valid_0's l2: 8.15223
[4192]	valid_0's l2: 8.15223
[4193]	valid_0's l2: 8.15216
[4194]	valid_0's l2: 8.15221
[4195]	valid_0's l2: 8.15199
[4196]	valid_0's l2: 8.1518
[4197]	valid_0's l2: 8.15144
[4198]	valid_0's l2: 8.15125
[4199]	valid_0's l2: 8.15137
[4200]	valid_0's l2: 8.15128
[4201]	valid_0's l2: 8.15146
[4202]	valid_0's l2: 8.15138
[4203]	valid_0's l2: 8.15136
[4204]	valid_0's l2: 8.15108
[4205]	valid_0's l2: 8.15094
[4206]	valid_0's l2: 8.15088
[4207]	valid_0's l2: 8.1506
[4208]	valid_0's l2: 8.15038
[4209]	valid_0's l2: 8.15015
[4210]	valid_0's l2: 8.15015
[4211]	valid_0's l2: 8.1501
[4212]	valid_0's l2: 8.14999
[4213]	valid_0's l2: 8.15016
[4214]	valid_0's l2: 8.14998
[4215]	valid_0's l2: 8.14974
[4216]	valid_0's l2: 8.14964
[4217]	valid_0's l2: 8.14966
[4218]	valid_0's l2: 8.14923
[4219]	valid_0's l2: 8.14911
[4220]	valid_0's l2: 8.14878
[4221]	valid_0's l2: 8.14862
[4222]	valid_0's l2: 8.14831
[4223]	valid_0's l2: 8.14822
[4224]	valid_0's 

LGBMRegressor(colsample_bytree=0.7520000000000001, feature_pre_filter=False,
              learning_rate=0.001, metric='mse', min_child_samples=1,
              n_estimators=10000000, num_leaves=127, objective='regression',
              random_state=42, reg_alpha=0.00010798877450667029,
              reg_lambda=0.572291122965791, subsample=0.8941758297667703,
              subsample_freq=1)

***
##  Departure

In [16]:
Data_d = Data_m.drop('ADR', axis=1)
Data_d = Data_d.drop('AAR', axis=1)
y = Data_raw.ADR.to_numpy()
X = Data_d.to_numpy()

#X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, y, test_size = 0.1, random_state = seed)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, y, test_size = 0.1, random_state = seed)
X_train_d, X_val_d, y_train_d, y_val_d = train_test_split(X_train_d, y_train_d, test_size=0.11, random_state = 13) 
    # val은 hyperparameter 검증에 사용
    # 0.11 x 0.9 = 0.099

################## 확실히 training data가 많을 때, 결과가 좋음  ##################

In [17]:
# Model fitting
reg_departure = LGBMRegressor(**params_d)

reg_departure.fit(X_train_d, y_train_d,
                  eval_set=[(X_val_d, y_val_d)],
                  eval_metric='l2',
                  early_stopping_rounds = 10)     # loss fuction이 n번 이상 좋아지지 않으면 멈춰라

[1]	valid_0's l2: 159.196
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 158.901
[3]	valid_0's l2: 158.603
[4]	valid_0's l2: 158.306
[5]	valid_0's l2: 158.022
[6]	valid_0's l2: 157.727
[7]	valid_0's l2: 157.428
[8]	valid_0's l2: 157.13
[9]	valid_0's l2: 156.837
[10]	valid_0's l2: 156.542
[11]	valid_0's l2: 156.245
[12]	valid_0's l2: 155.951
[13]	valid_0's l2: 155.66
[14]	valid_0's l2: 155.366
[15]	valid_0's l2: 155.073
[16]	valid_0's l2: 154.78
[17]	valid_0's l2: 154.488
[18]	valid_0's l2: 154.195
[19]	valid_0's l2: 153.902
[20]	valid_0's l2: 153.612
[21]	valid_0's l2: 153.322
[22]	valid_0's l2: 153.031
[23]	valid_0's l2: 152.744
[24]	valid_0's l2: 152.453
[25]	valid_0's l2: 152.167
[26]	valid_0's l2: 151.883
[27]	valid_0's l2: 151.598
[28]	valid_0's l2: 151.314
[29]	valid_0's l2: 151.032
[30]	valid_0's l2: 150.749
[31]	valid_0's l2: 150.464
[32]	valid_0's l2: 150.18
[33]	valid_0's l2: 149.91
[34]	valid_0's l2: 149.633
[35]	valid_0's l2: 149.359
[36]	val

[308]	valid_0's l2: 90.4745
[309]	valid_0's l2: 90.3108
[310]	valid_0's l2: 90.148
[311]	valid_0's l2: 89.9846
[312]	valid_0's l2: 89.8218
[313]	valid_0's l2: 89.6593
[314]	valid_0's l2: 89.4956
[315]	valid_0's l2: 89.3341
[316]	valid_0's l2: 89.1736
[317]	valid_0's l2: 89.0117
[318]	valid_0's l2: 88.8513
[319]	valid_0's l2: 88.6935
[320]	valid_0's l2: 88.5341
[321]	valid_0's l2: 88.3739
[322]	valid_0's l2: 88.2159
[323]	valid_0's l2: 88.0569
[324]	valid_0's l2: 87.9028
[325]	valid_0's l2: 87.7438
[326]	valid_0's l2: 87.5854
[327]	valid_0's l2: 87.4347
[328]	valid_0's l2: 87.2992
[329]	valid_0's l2: 87.1423
[330]	valid_0's l2: 86.9853
[331]	valid_0's l2: 86.8285
[332]	valid_0's l2: 86.6723
[333]	valid_0's l2: 86.5173
[334]	valid_0's l2: 86.3625
[335]	valid_0's l2: 86.2072
[336]	valid_0's l2: 86.0514
[337]	valid_0's l2: 85.8953
[338]	valid_0's l2: 85.741
[339]	valid_0's l2: 85.5858
[340]	valid_0's l2: 85.4323
[341]	valid_0's l2: 85.2782
[342]	valid_0's l2: 85.1268
[343]	valid_0's l2: 84

[606]	valid_0's l2: 53.854
[607]	valid_0's l2: 53.7634
[608]	valid_0's l2: 53.6724
[609]	valid_0's l2: 53.582
[610]	valid_0's l2: 53.4917
[611]	valid_0's l2: 53.4022
[612]	valid_0's l2: 53.3152
[613]	valid_0's l2: 53.2247
[614]	valid_0's l2: 53.1371
[615]	valid_0's l2: 53.0472
[616]	valid_0's l2: 52.9579
[617]	valid_0's l2: 52.8703
[618]	valid_0's l2: 52.7811
[619]	valid_0's l2: 52.6919
[620]	valid_0's l2: 52.6051
[621]	valid_0's l2: 52.5157
[622]	valid_0's l2: 52.4281
[623]	valid_0's l2: 52.3402
[624]	valid_0's l2: 52.2552
[625]	valid_0's l2: 52.1668
[626]	valid_0's l2: 52.078
[627]	valid_0's l2: 51.99
[628]	valid_0's l2: 51.9051
[629]	valid_0's l2: 51.8185
[630]	valid_0's l2: 51.7308
[631]	valid_0's l2: 51.6431
[632]	valid_0's l2: 51.5562
[633]	valid_0's l2: 51.4698
[634]	valid_0's l2: 51.4396
[635]	valid_0's l2: 51.3538
[636]	valid_0's l2: 51.2689
[637]	valid_0's l2: 51.1825
[638]	valid_0's l2: 51.0961
[639]	valid_0's l2: 51.0112
[640]	valid_0's l2: 50.9259
[641]	valid_0's l2: 50.83

[908]	valid_0's l2: 33.237
[909]	valid_0's l2: 33.1875
[910]	valid_0's l2: 33.137
[911]	valid_0's l2: 33.0871
[912]	valid_0's l2: 33.0373
[913]	valid_0's l2: 32.9878
[914]	valid_0's l2: 32.9379
[915]	valid_0's l2: 32.8885
[916]	valid_0's l2: 32.839
[917]	valid_0's l2: 32.7902
[918]	valid_0's l2: 32.7408
[919]	valid_0's l2: 32.6908
[920]	valid_0's l2: 32.6413
[921]	valid_0's l2: 32.5939
[922]	valid_0's l2: 32.5446
[923]	valid_0's l2: 32.4949
[924]	valid_0's l2: 32.4462
[925]	valid_0's l2: 32.3976
[926]	valid_0's l2: 32.3489
[927]	valid_0's l2: 32.2999
[928]	valid_0's l2: 32.2509
[929]	valid_0's l2: 32.202
[930]	valid_0's l2: 32.1532
[931]	valid_0's l2: 32.1044
[932]	valid_0's l2: 32.0565
[933]	valid_0's l2: 32.0085
[934]	valid_0's l2: 31.9601
[935]	valid_0's l2: 31.9288
[936]	valid_0's l2: 31.8816
[937]	valid_0's l2: 31.8347
[938]	valid_0's l2: 31.7864
[939]	valid_0's l2: 31.7385
[940]	valid_0's l2: 31.6906
[941]	valid_0's l2: 31.6434
[942]	valid_0's l2: 31.596
[943]	valid_0's l2: 31.54

[1228]	valid_0's l2: 21.3112
[1229]	valid_0's l2: 21.2851
[1230]	valid_0's l2: 21.2604
[1231]	valid_0's l2: 21.2329
[1232]	valid_0's l2: 21.2057
[1233]	valid_0's l2: 21.1787
[1234]	valid_0's l2: 21.1523
[1235]	valid_0's l2: 21.1256
[1236]	valid_0's l2: 21.0995
[1237]	valid_0's l2: 21.0727
[1238]	valid_0's l2: 21.0457
[1239]	valid_0's l2: 21.0189
[1240]	valid_0's l2: 20.9922
[1241]	valid_0's l2: 20.9655
[1242]	valid_0's l2: 20.9392
[1243]	valid_0's l2: 20.9129
[1244]	valid_0's l2: 20.8869
[1245]	valid_0's l2: 20.8618
[1246]	valid_0's l2: 20.8355
[1247]	valid_0's l2: 20.8095
[1248]	valid_0's l2: 20.7842
[1249]	valid_0's l2: 20.7588
[1250]	valid_0's l2: 20.7328
[1251]	valid_0's l2: 20.7065
[1252]	valid_0's l2: 20.6805
[1253]	valid_0's l2: 20.6555
[1254]	valid_0's l2: 20.6297
[1255]	valid_0's l2: 20.6041
[1256]	valid_0's l2: 20.5779
[1257]	valid_0's l2: 20.5521
[1258]	valid_0's l2: 20.5266
[1259]	valid_0's l2: 20.5008
[1260]	valid_0's l2: 20.4756
[1261]	valid_0's l2: 20.4529
[1262]	valid_0

...

[3330]	valid_0's l2: 7.6113
[3331]	valid_0's l2: 7.61056
[3332]	valid_0's l2: 7.61015
[3333]	valid_0's l2: 7.60947
[3334]	valid_0's l2: 7.60902
[3335]	valid_0's l2: 7.60884
[3336]	valid_0's l2: 7.60842
[3337]	valid_0's l2: 7.6079
[3338]	valid_0's l2: 7.60741
[3339]	valid_0's l2: 7.60712
[3340]	valid_0's l2: 7.60666
[3341]	valid_0's l2: 7.60604
[3342]	valid_0's l2: 7.60575
[3343]	valid_0's l2: 7.60519
[3344]	valid_0's l2: 7.6047
[3345]	valid_0's l2: 7.6042
[3346]	valid_0's l2: 7.60354
[3347]	valid_0's l2: 7.60296
[3348]	valid_0's l2: 7.60282
[3349]	valid_0's l2: 7.60225
[3350]	valid_0's l2: 7.60179
[3351]	valid_0's l2: 7.60112
[3352]	valid_0's l2: 7.60062
[3353]	valid_0's l2: 7.60034
[3354]	valid_0's l2: 7.5999
[3355]	valid_0's l2: 7.5995
[3356]	valid_0's l2: 7.59871
[3357]	valid_0's l2: 7.59812
[3358]	valid_0's l2: 7.59785
[3359]	valid_0's l2: 7.59725
[3360]	valid_0's l2: 7.59688
[3361]	valid_0's l2: 7.59632
[3362]	valid_0's l2: 7.59603
[3363]	valid_0's l2: 7.59551
[3364]	valid_0's l2:

[3636]	valid_0's l2: 7.48515
[3637]	valid_0's l2: 7.48498
[3638]	valid_0's l2: 7.48464
[3639]	valid_0's l2: 7.48475
[3640]	valid_0's l2: 7.48446
[3641]	valid_0's l2: 7.48442
[3642]	valid_0's l2: 7.48446
[3643]	valid_0's l2: 7.48428
[3644]	valid_0's l2: 7.48406
[3645]	valid_0's l2: 7.4837
[3646]	valid_0's l2: 7.48339
[3647]	valid_0's l2: 7.48313
[3648]	valid_0's l2: 7.48272
[3649]	valid_0's l2: 7.48242
[3650]	valid_0's l2: 7.48175
[3651]	valid_0's l2: 7.48144
[3652]	valid_0's l2: 7.48129
[3653]	valid_0's l2: 7.48097
[3654]	valid_0's l2: 7.48049
[3655]	valid_0's l2: 7.48019
[3656]	valid_0's l2: 7.48012
[3657]	valid_0's l2: 7.4799
[3658]	valid_0's l2: 7.47965
[3659]	valid_0's l2: 7.47944
[3660]	valid_0's l2: 7.47883
[3661]	valid_0's l2: 7.47845
[3662]	valid_0's l2: 7.47807
[3663]	valid_0's l2: 7.47756
[3664]	valid_0's l2: 7.47728
[3665]	valid_0's l2: 7.47682
[3666]	valid_0's l2: 7.47656
[3667]	valid_0's l2: 7.47633
[3668]	valid_0's l2: 7.47596
[3669]	valid_0's l2: 7.47586
[3670]	valid_0's

[3964]	valid_0's l2: 7.40848
[3965]	valid_0's l2: 7.40834
[3966]	valid_0's l2: 7.40794
[3967]	valid_0's l2: 7.40757
[3968]	valid_0's l2: 7.40746
[3969]	valid_0's l2: 7.40723
[3970]	valid_0's l2: 7.40705
[3971]	valid_0's l2: 7.40674
[3972]	valid_0's l2: 7.40641
[3973]	valid_0's l2: 7.40605
[3974]	valid_0's l2: 7.40599
[3975]	valid_0's l2: 7.40571
[3976]	valid_0's l2: 7.40565
[3977]	valid_0's l2: 7.40562
[3978]	valid_0's l2: 7.40553
[3979]	valid_0's l2: 7.40537
[3980]	valid_0's l2: 7.40527
[3981]	valid_0's l2: 7.40497
[3982]	valid_0's l2: 7.40469
[3983]	valid_0's l2: 7.40466
[3984]	valid_0's l2: 7.40428
[3985]	valid_0's l2: 7.40381
[3986]	valid_0's l2: 7.40365
[3987]	valid_0's l2: 7.40343
[3988]	valid_0's l2: 7.40318
[3989]	valid_0's l2: 7.403
[3990]	valid_0's l2: 7.40284
[3991]	valid_0's l2: 7.40251
[3992]	valid_0's l2: 7.40236
[3993]	valid_0's l2: 7.40221
[3994]	valid_0's l2: 7.40205
[3995]	valid_0's l2: 7.40202
[3996]	valid_0's l2: 7.40172
[3997]	valid_0's l2: 7.40175
[3998]	valid_0's

[4265]	valid_0's l2: 7.36626
[4266]	valid_0's l2: 7.36637
[4267]	valid_0's l2: 7.36626
[4268]	valid_0's l2: 7.36622
[4269]	valid_0's l2: 7.36621
[4270]	valid_0's l2: 7.36618
[4271]	valid_0's l2: 7.36615
[4272]	valid_0's l2: 7.36584
[4273]	valid_0's l2: 7.36564
[4274]	valid_0's l2: 7.36542
[4275]	valid_0's l2: 7.36526
[4276]	valid_0's l2: 7.36513
[4277]	valid_0's l2: 7.36512
[4278]	valid_0's l2: 7.36498
[4279]	valid_0's l2: 7.36495
[4280]	valid_0's l2: 7.36472
[4281]	valid_0's l2: 7.36445
[4282]	valid_0's l2: 7.36422
[4283]	valid_0's l2: 7.36426
[4284]	valid_0's l2: 7.36414
[4285]	valid_0's l2: 7.36394
[4286]	valid_0's l2: 7.36371
[4287]	valid_0's l2: 7.36364
[4288]	valid_0's l2: 7.36366
[4289]	valid_0's l2: 7.36361
[4290]	valid_0's l2: 7.36351
[4291]	valid_0's l2: 7.36352
[4292]	valid_0's l2: 7.36336
[4293]	valid_0's l2: 7.36324
[4294]	valid_0's l2: 7.36337
[4295]	valid_0's l2: 7.36311
[4296]	valid_0's l2: 7.36313
[4297]	valid_0's l2: 7.36291
[4298]	valid_0's l2: 7.36301
[4299]	valid_0

LGBMRegressor(colsample_bytree=0.7, feature_pre_filter=False,
              learning_rate=0.001, metric='mse', min_child_samples=1,
              n_estimators=10000000, num_leaves=127, objective='regression',
              random_state=42, reg_alpha=3.2041046397019426e-05,
              reg_lambda=2.4947477155359143e-05, subsample=0.8862849544828171,
              subsample_freq=1)

***

## Cross validation

for loop안에서 predict하고 평균을 내야 함 <br>
-> demand를 하나씩 증가시키리면, 바로 CV를 사용하기는 어려울 듯

In [ ]:
"""



n_fold = 10

cv = KFold(n_splits=n_fold, shuffle=True, random_state=seed)


Data_a = Data_m.drop('AAR', axis=1)
y_a = Data_raw.AAR.to_numpy()
X_a = Data_a.to_numpy()
Data_d = Data_m.drop('ADR', axis=1)
y_d = Data_raw.ADR.to_numpy()
X_d = Data_d.to_numpy()

p_val_a = np.zeros(X_a.shape[0])
predict_a = np.zeros(X_a.shape[0])
p_val_d = np.zeros(X_d.shape[0])
predict_d = np.zeros(X_d.shape[0])

for i, (i_trn, i_val) in enumerate(cv.split(X_a, y_a), 1):        # 몇번째인지 보기 위해 enumerate 사용
    print('----------------------------------------------------------------------------')
    print(f'Training model for Cross-Validation #{i}')
    reg_arrival = LGBMRegressor(**params)    #############params_a
    reg_arrival.fit(X_a[i_trn], y_a[i_trn],
                sample_weight = None,                   # Weights of training data
                init_score = None,                      # Weights of training data
                eval_set = [(X_a[i_val], y_a[i_val])],  # pairs to use as validation sets
                eval_sample_weight = None,              # Weights of eval data
                eval_init_score = None,                 # Init score of eval data.   
                eval_metric = 'l2',                     # Default: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier, ‘ndcg’ for LGBMRanker
                early_stopping_rounds = 10 )            # loss fuction이 n번 이상 좋아지지 않으면 멈춰라
    
    p_val_a[i_val] = reg_arrival.predict(X_a[i_val])
    predict_a += reg_arrival.predict(X_a) / n_fold      ##### 이 자리에 원래는 test data가 들어가면 됨
                                                        # test data의 prediction은 CV에서 각 dataset이 예측한 값들의 평균이므로
    
for i, (i_trn, i_val) in enumerate(cv.split(X_d, y_d), 1):        # 몇번째인지 보기 위해 enumerate 사용
    print('----------------------------------------------------------------------------')
    print(f'Training model for Cross-Validation #{i}')
    reg_departure = LGBMRegressor(**params)    #############params_d
    reg_departure.fit(X_d[i_trn], y_d[i_trn],
                      sample_weight = None,                   # Weights of training data
                      init_score = None,                      # Weights of training data
                      eval_set = [(X_d[i_val], y_d[i_val])],  # pairs to use as validation sets
                      eval_sample_weight = None,              # Weights of eval data
                      eval_init_score = None,                 # Init score of eval data.   
                      eval_metric = 'l2',                     # Default: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier, ‘ndcg’ for LGBMRanker
                      early_stopping_rounds = 10 )            # loss fuction이 n번 이상 좋아지지 않으면 멈춰라
    
    p_val_d[i_val] = reg_departure.predict(X_d[i_val])
    predict_d += reg_departure.predict(X_d) / n_fold          ##### 이 자리에 원래는 test data가 들어가면 됨
                                                              # test data의 prediction은 CV에서 각 dataset이 예측한 값들의 평균이므로
 
    
    
"""

In [ ]:
"""
print(f'Arrival RMSE : {np.sqrt(mean_squared_error(y_a, predict_a)):.4f}')
print(f'Arrival Training R^2 : {r2_score(y_a, predict_a) * 100:.4f}')
print(f'Departure RMSE : {np.sqrt(mean_squared_error(y_d, predict_d)):.4f}')
print(f'Departure Training R^2 : {r2_score(y_d, predict_d) * 100:.4f}')

np.savetxt('p_val_a.csv', p_val_a, fmt='%.6f', delimiter=',')
np.savetxt('p_val_d.csv', p_val_d, fmt='%.6f', delimiter=',')
np.savetxt('predict_a.csv', predict_a, fmt='%.6f', delimiter=',')
np.savetxt('predict_d.csv', predict_d, fmt='%.6f', delimiter=',')
"""

이렇게 각각의 model의(e.g. lightGBM, XGBOOST, Randomforest ...) Cross Validation을 예측한 결과와, Test data에 대해 예측한 결과를 파일로 저장 <br>
-> 다음 stage에서는 CV결과들을 input data로 사용, 기존의 label은 그대로 사용, test data의 예측값을 다음 stage에서 test data의 input으로 사용 <br>
-> 계속 쌓아갈 수 있음 (=Stacking, 보통 stage 1,2정도면 이후 효과는 미비)


stacking, ensemble 보다 feature engineering, hyperparameter tuning의 성능 향상이 훨씬 커서 앞의 방법은 굳이 사용하지 않아도 괜찮음

## 왔다갔다 - 이것 좀 만들어봐

In [ ]:
#################################################################################################################
#################################################################################################################
#################################################################################################################
#################################################################################################################
#################################################################################################################



# Arrival data
Data_a = Data_m.drop('AAR', axis=1)
Data_a = Data_a.drop('ADR', axis=1)
# Data_a = Data_a.join(              )     ADR prediction을 넣기

y = Data_raw['AAR'].to_numpy()
X = Data_a.to_numpy()

# Model fitting
reg_arrival = LGBMRegressor(**params)
reg_arrival.fit(X, y,
                sample_weight = None,                   # Weights of training data
                init_score = None,                      # Weights of training data
                eval_set = [(X_val_a, y_val_a)],        # pairs to use as validation sets
                eval_sample_weight = None,              # Weights of eval data
                eval_init_score = None,                 # Init score of eval data.   
                eval_metric = 'l2',                     # Default: ‘l2’ for LGBMRegressor, ‘logloss’ for LGBMClassifier, ‘ndcg’ for LGBMRanker
                early_stopping_rounds = 10 )            # loss fuction이 n번 이상 좋아지지 않으면 멈춰라

# Arrival prediction





# Departure data
Data_d = Data_m.drop('ADR', axis=1)
Data_d = Data_d.drop('AAR', axis=1)
# Data_d = Data_d.join(              )     ADR prediction을 넣기

y = Data_raw.ADR.to_numpy()
X = Data_d.to_numpy()

# Model fitting
reg_departure = LGBMRegressor(**params_d)
reg_departure.fit(X_train_d, y_train_d,
                  eval_set=[(X_val_d, y_val_d)],
                  eval_metric='l2',
                  early_stopping_rounds = 10)     # loss fuction이 n번 이상 좋아지지 않으면 멈춰라


# departure prediction



#################################################################################################################
#################################################################################################################
#################################################################################################################
#################################################################################################################
#################################################################################################################

***
***
***

# Evaluation

<sklearn.metrics> <br>

* regression <br>
mean_squared_error, mean_absolute_error, r2_score <br><br>

* classification <br>
log_loss, roc_auc_score, accuracy_score, confusion_matrix <br>

In [18]:
def Test(Data_raw, ops='arrival', start=0, end = 10):        # ops : arrival , departure
    if ops == 'arrival':
        Data_raw = Data_raw.drop('ADR', axis=1)
        datelist = pd.DataFrame(pd.date_range('2019-01-01', '2019-12-31 23:00', freq = '1h'))
        test_result = pd.DataFrame({'Date' : datelist[start:end][0],
                                    'EAD' : Data_raw['EAD'][start:end],
                                    'Actual AAR' : Data_raw['AAR'][start:end], 
                                    'Predicted AAR' : reg_arrival.predict(Data_raw.drop('AAR', axis=1)[start:end]), 
                                    'Difference' : reg_arrival.predict(Data_raw.drop('AAR', axis=1)[start:end]) - Data_raw['AAR'][start:end]})
    elif ops == 'departure':
        Data_raw = Data_raw.drop('AAR', axis=1)        
        datelist = pd.DataFrame(pd.date_range('2019-01-01', '2019-12-31 23:00', freq = '1h'))
        test_result = pd.DataFrame({'Date' : datelist[start:end][0],
                                    'EDD' : Data_raw['EDD'][start:end],
                                    'Actual ADR' : Data_raw['ADR'][start:end], 
                                    'Predicted ADR' : reg_departure.predict(Data_raw.drop('ADR', axis=1)[start:end]), 
                                    'Difference' : reg_departure.predict(Data_raw.drop('ADR', axis=1)[start:end]) - Data_raw['ADR'][start:end]})
        
    return test_result

***
## Arrival

In [19]:
# Predict & Evaluation
print(f'RMSE : {np.sqrt(mean_squared_error(y_train_a, reg_arrival.predict(X_train_a))):.4f}')
print(f'Training R^2 : {r2_score(y_train_a, reg_arrival.predict(X_train_a)) * 100:.4f}')
print(f'Test R^2 : {r2_score(y_test_a, reg_arrival.predict(X_test_a)) * 100:.4f}')

RMSE : 1.6372
Training R^2 : 97.8511
Test R^2 : 94.1275


In [ ]:
# Feature Importance
imp = pd.DataFrame({'feature': Data_a.columns, 'importance': reg_arrival.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh', figsize = (20,20))
plt.legend(loc='lower right')

In [ ]:
plot_metric(reg_arrival, figsize=(10,10))

In [ ]:
# Test
Test_all = Test(Data_raw, ops='arrival', start = 0, end = 8760).sort_values('Difference')
Test_all['Difference'].abs().sort_values(ascending=False).head(30)

-------------------
## Departure

In [20]:
# Predict & Evaluation
print(f'RMSE : {np.sqrt(mean_squared_error(y_train_d, reg_departure.predict(X_train_d))):.4f}')
print(f'Training R^2 : {r2_score(y_train_d, reg_departure.predict(X_train_d)) * 100:.4f}')
print(f'Test R^2 : {r2_score(y_test_d, reg_departure.predict(X_test_d)) * 100:.4f}')

RMSE : 1.4181
Training R^2 : 98.6946
Test R^2 : 95.9547


In [ ]:
# Feature Importance
imp = pd.DataFrame({'feature': Data_d.columns, 'importance': reg_departure.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh', figsize = (20,20))
plt.legend(loc='lower right')

In [ ]:
plot_metric(reg_departure, figsize=(10,10))

In [ ]:
# Test
Test_all = Test(Data_raw, ops='departure', start = 0, end = 8760).sort_values('Difference')
Test_all['Difference'].abs().sort_values(ascending=False).head(30)

***
## Additional

In [ ]:
############################################################
# Arrival
############################################################

pred_test_a = Data_m.drop('ADR', axis=1)
pred_test_a['pred'] = reg_arrival.predict(pred_test_a.drop('AAR', axis=1))
sns.pairplot(data=pred_test_a, vars=['pred', 'AAR'], size=10, plot_kws={'alpha': .5})

In [ ]:
############################################################
# Departure
############################################################

pred_test_d = Data_m.drop('AAR', axis=1)
pred_test_d['pred'] = reg_departure.predict(pred_test_d.drop('ADR', axis=1))
sns.pairplot(data=pred_test_d, vars=['pred', 'ADR'], size=10, plot_kws={'alpha': .5})

***
***
***

# Maximum Capacity

In [ ]:
###################################################################################################################


# 원래 demand부터 시작 -> extra demand를 더함


###################################################################################################################


def max_capacity(Data_raw, Data_a, Data_d, example):    # 50까지 늘림
    
    # extra
    extra_demand = 50
    
    # arrival
    data_a = Data_a.to_numpy()[example:example+1]
    XX_a = np.zeros((1,len(data_a.T)))
    for i in range(extra_demand+1):
        XX_a = np.append(XX_a, data_a, axis = 0)
        XX_a[i,0] = XX_a[i,0] + i-1
    XXX_a = XX_a[1:extra_demand+1]
    XXXX_a = np.arange(XXX_a[0,0], XXX_a[extra_demand-1,0]+1, 1)
    YYYY_a = reg_arrival.predict(XXX_a[0:extra_demand+1])
    max_aar = float(max(YYYY_a))
    actual_aar = int(Data_raw['AAR'][example:example+1])
    ead = int(Data_raw['EAD'][example:example+1])
    prediction_a = float(reg_arrival.predict(Data_a[example:example+1]))
    
    # departure
    data_d = Data_d.to_numpy()[example:example+1]
    XX_d = np.zeros((1,len(data_d.T)))
    for i in range(extra_demand+1):
        XX_d = np.append(XX_d, data_d, axis = 0)
        XX_d[i,2] = XX_d[i,2] + i-1
    XXX_d = XX_d[1:extra_demand+1]
    XXXX_d = np.arange(XXX_d[0,2], XXX_d[extra_demand-1,2]+1, 1)
    YYYY_d = reg_departure.predict(XXX_d[0:extra_demand+1])
    max_adr = float(max(YYYY_d))
    actual_adr = int(Data_raw['ADR'][example:example+1])
    edd = int(Data_raw['EDD'][example:example+1])
    prediction_d = float(reg_departure.predict(Data_d[example:example+1])) 

    # Total
    result_ = pd.DataFrame({'EAD' : XXXX_a ,'Predicted AAR' : YYYY_a, 'EDD' : XXXX_d ,'Predicted ADR' : YYYY_d, 
                            'Demand' : XXXX_a + YYYY_a, 'Capacity' : YYYY_a + YYYY_d})
    
    # plot
    plt.figure(figsize=(15,15))
    plt.title('Maximum Capacity', fontsize=30)
    plt.xlabel('Estimated Demands', fontsize=25)
    plt.ylabel('Capacity', fontsize=25)
    plt.plot(XXXX_a, YYYY_a, label = 'Arrival')
    plt.plot(XXXX_d, YYYY_d, label = 'Departure')
    plt.plot(XXXX_a[:-15]+XXXX_d[:-15], YYYY_a[:-15] + YYYY_d[:-15], label = 'Total')   # 안 예뻐서 뒤에 15개 자름
    plt.legend(prop={'size': 20})
    ax = plt.subplot()
    plt.text(0.95, 0.04,    # 위치조정
             f"""
             * Predicted Max AAR: {max_aar:.3f}\n
               Predicted AAR : {prediction_a:.3f}\n
               Actual AAR : {actual_aar} \n               
               EAD : {ead} \n\n
             
             * Predicted Max ADR: {max_adr:.3f}\n
               Predicted ADR : {prediction_d:.3f}\n
               Actual ADR : {actual_adr} \n               
               EDD : {edd} \n\n
             
             * Predicted Max Capacity: {max_aar + max_adr:.3f}\n
               Predicted Rate : {prediction_a + prediction_d:.3f}\n
               Actual Rate : {actual_aar + actual_adr} \n               
               Demand : {ead + edd}'
             """,
             fontsize=15, style='italic', transform=ax.transAxes, bbox={'facecolor': 'grey', 'alpha': 0, 'pad': 5})
    plt.show()    
    return result_

In [ ]:
###################################################################################################################


# 0부터 시작

###################################################################################################################

def max_capacity(Data_raw, Data_a, Data_d, example):    # 50까지 늘림
    
    # extra
    demand = 75
    
    # arrival
    data_a = Data_a.to_numpy()[example:example+1]
    XX_a = np.zeros((1,len(data_a.T)))
    
    original_demand_a = int(data_a[0][0])
    for i in range(0,demand+1):
        XX_a = np.append(XX_a, data_a, axis = 0)
        XX_a[i,0] = XX_a[i,0] + i - original_demand_a

        
        
    XXX_a = XX_a[1:demand+1]
    XXXX_a = np.arange(XXX_a[0,0], XXX_a[demand-1,0]+1, 1)
    YYYY_a = reg_arrival.predict(XXX_a[0:demand])
    max_aar = float(max(YYYY_a))
    actual_aar = int(Data_raw.drop('ADR', axis=1)['AAR'][example:example+1])
    ead = int(Data_raw.drop('ADR', axis=1)['EAD'][example:example+1])
    prediction_a = float(reg_arrival.predict(Data_a[example:example+1]))
    
    # departure
    data_d = Data_d.to_numpy()[example:example+1]
    XX_d = np.zeros((1,len(data_d.T)))
    
    original_demand_d = int(data_d[0][1])
    for i in range(0,demand+1):
        XX_d = np.append(XX_d, data_d, axis = 0)
        XX_d[i,1] = XX_d[i,1] + i - original_demand_d
    
        
        
    XXX_d = XX_d[1:demand+1]
    XXXX_d = np.arange(XXX_d[0,1], XXX_d[demand-1,1]+1, 1)
    YYYY_d = reg_departure.predict(XXX_d[0:demand])
    max_adr = float(max(YYYY_d))
    actual_adr = int(Data_raw.drop('AAR', axis=1)['ADR'][example:example+1])
    edd = int(Data_raw.drop('AAR', axis=1)['EDD'][example:example+1])
    prediction_d = float(reg_departure.predict(Data_d[example:example+1])) 

    # Total
    #result_ = pd.DataFrame({'EAD' : XXXX_a ,'Predicted AAR' : YYYY_a, 
    #                        'EDD' : XXXX_d ,'Predicted ADR' : YYYY_d, 
    #                        'Demand' : XXXX_a + YYYY_a, 'Capacity' : YYYY_a + YYYY_d})
    
    # plot
    plt.figure(figsize=(15,15))
    plt.title('Maximum Capacity', fontsize=30)
    plt.xlabel('Estimated Demands', fontsize=25)
    plt.ylabel('Capacity', fontsize=25)
    plt.plot(XXXX_a, YYYY_a, label = 'Arrival')
    plt.plot(XXXX_d, YYYY_d, label = 'Departure')
    plt.plot(XXXX_a[:-15]+XXXX_d[:-15], YYYY_a[:-15] + YYYY_d[:-15], label = 'Total')   # 안 예뻐서 뒤에 15개 자름
    plt.legend(prop={'size': 20})
    ax = plt.subplot()
    plt.text(0.95, 0.04,    # 위치조정
             f"""
             * Predicted Max AAR: {max_aar:.3f}\n
               Predicted AAR : {prediction_a:.3f}\n
               Actual AAR : {actual_aar} \n               
               EAD : {ead} \n\n
             
             * Predicted Max ADR: {max_adr:.3f}\n
               Predicted ADR : {prediction_d:.3f}\n
               Actual ADR : {actual_adr} \n               
               EDD : {edd} \n\n
             
             * Predicted Max Capacity: {max_aar + max_adr:.3f}\n
               Predicted Rate : {prediction_a + prediction_d:.3f}\n
               Actual Rate : {actual_aar + actual_adr} \n               
               Demand : {ead + edd}'
             """,
             fontsize=15, style='italic', transform=ax.transAxes, bbox={'facecolor': 'grey', 'alpha': 0, 'pad': 5})
    plt.show()    
    return #result_

In [ ]:
# 원하는 시간의 Max Capacity 그래프

max_capacity(Data_raw, Data_a, Data_d, 0)     # 숫자에 원하는 data번호 넣기

< Notable Samples> 

* pd.DataFrame(Data_raw).idxmax()             # 각 column별 max값의 index

AAR 최소 : 2 <br>
AAR 최대 : 1481 <br>
EAD 최대 : 641 <br>
ADR 최소 : 28 <br>
ADR 최대 : 538 <br>
EDD 최대 : 273 <br>
Total Rate(AAR+ADR) 최대(77) : 5703 <br>
Total Demand(EAD+EDD) 최대(70) : 4814 <br><br>

Arrival_reaminder 최대 : 5992 <br>
Departure_remainder 최대 : 8242 <br><br>

WSPD 최대 : 5990 <br>
W_GST 최대 : 5990 <br>
시정 최소 : 1755 <br>
Ceiling 최소 : 321 <br>
RVR 최소 1448 <br><br>

TAF WSPD 최대 : 5988 <br>
TAF W_GST 최대 : 5988 <br>
TAF 시정 최소 : 319 <br>
TAF Ceiling 최소 : 10?? <br>
<br>
<br>
* Max capacity에 영향을 주는 것 <br>
1. 관제사 워크로드 <br>
2. 기상 <br>
3. 항공기 성능(어떤 어프로치 가능한지 등) <br>


In [ ]:
# Data : 불러온 파일
# Data_temp : 일단 필요 없어 보이는 column을 버린 것
# data_taf : dictinoary 안에 Data_6 - Data_24 전부 넣은 것
# Data_6, Data_12 .. : 해당 시간 전의 TAF만 있고, 나머지 시간의 TAF는 버린 것
# Data_m : Data_# 과 동일
# Data_raw : Data_#과 동일

# Data_a : Data_m에서 AAR, ADR 뺀 것
# Data_d : Data_m에서 ADR, AAR 뺸 것

***
***
***

# 추가 실험

***
***
***

# 모델 완성 후 할 일

* data.py / training.py / evaluation.py 등 이렇게 나누기

***
***
***

# 모델 저장

In [21]:
import joblib

lgbr_a = ".//test//lgbr_a.pkl"
lgbr_d = ".//test//lgbr_d.pkl"
joblib.dump(reg_arrival, lgbr_a)
joblib.dump(reg_departure, lgbr_d)

['.//test//lgbr_d.pkl']